<a href="https://colab.research.google.com/github/AlejandroHerrera15/Caso_estudio_salud/blob/main/g_Modelos_complemento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El tunning se realizo en colab ya que visual estaba presentando problemas con el directorio

In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.3 MB/s eta 0:00:00


Lectura de datos

NOTA: Los dos primeros tunning estan realizados con una dimension de las imagenes de 120x120 pixeles en tres canales

In [2]:
import numpy as np
import joblib ### para cargar array
########Paquetes para NN #########
import tensorflow as tf
from sklearn import metrics ### para analizar modelo
import pandas as pd
####instalar paquete !pip install keras-tuner
import keras_tuner as kt

### cargar bases_procesadas ####

x_train = joblib.load('x_train.pkl')
y_train = joblib.load('y_train.pkl')
x_test = joblib.load('x_test.pkl')
y_test = joblib.load('y_test.pkl')

x_train[0]
x_train=x_train.astype("float32")
x_test=x_test.astype("float32")
x_train.max()
x_train.min()

x_train/=255
x_test/=255

x_train.shape
x_test.shape

np.product(x_train[15].shape) ## cantidad de variables por imagen

np.unique(y_train, return_counts=True)
np.unique(y_test, return_counts=True)

FileNotFoundError: [Errno 2] No such file or directory: 'x_train.pkl'

In [ ]:
display(x_train.shape)
display(x_test.shape)
display(np.product(x_train[15].shape))

(4070, 120, 120, 3)

(694, 120, 120, 3)

43200

Modelo con tunning 1

In [ ]:
##### función con definicion de hiperparámetros a afinar
hp = kt.HyperParameters()
def build_model(hp):

    dropout_rate=hp.Float('DO', min_value=0.1, max_value= 0.5, step=0.05)
    reg_strength = hp.Float("rs", min_value=0.001, max_value=0.005, step=0.001)
    optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop']) ### en el contexto no se debería afinar

    #filtros1 = hp.Int("filtros1", min_value=16, max_value=128, step=32)
    #filtros2 = hp.Int("filtros2", min_value=32, max_value=128, step=32)
    #filtros3 = hp.Int("filtros3", min_value=64, max_value=128, step=32)

    ####hp.Int
    ####hp.Choice


    model= tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=x_train.shape[1:], kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])



    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=0.001)
    else:
        opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)

    model.compile(
        optimizer=opt, loss="binary_crossentropy",  metrics=["Precision"],
    )


    return model



###########

tuner = kt.RandomSearch(
    hypermodel=build_model,
    hyperparameters=hp,
    tune_new_entries=True,
    objective=kt.Objective("precision", direction="max"),
    max_trials=10,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)


tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test), batch_size=100)

fc_best_model = tuner.get_best_models(num_models=1)[0]
tuner.results_summary()
fc_best_model.summary()


#################### Mejor redes ##############
#fc_best_model.save('salidas\\fc_model.h5')
#cnn_model.save('salidas\\cnn_model.h5')


#cargar_modelo=tf.keras.models.load_model('salidas\\cnn_model.h5')
#cargar_modelo.summary()

Trial 10 Complete [00h 07m 26s]
precision: 0.7274826765060425

Best precision So Far: 0.9397445917129517
Total elapsed time: 01h 04m 52s
Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="precision", direction="max")

Trial 06 summary
Hyperparameters:
DO: 0.30000000000000004
rs: 0.004
optimizer: adam
Score: 0.9397445917129517

Trial 00 summary
Hyperparameters:
DO: 0.4
rs: 0.002
optimizer: rmsprop
Score: 0.933386504650116

Trial 08 summary
Hyperparameters:
DO: 0.30000000000000004
rs: 0.002
optimizer: rmsprop
Score: 0.9190267324447632

Trial 03 summary
Hyperparameters:
DO: 0.2
rs: 0.004
optimizer: rmsprop
Score: 0.9186413884162903

Trial 04 summary
Hyperparameters:
DO: 0.45000000000000007
rs: 0.001
optimizer: rmsprop
Score: 0.894385576248169

Trial 07 summary
Hyperparameters:
DO: 0.15000000000000002
rs: 0.001
optimizer: sgd
Score: 0.7908182144165039

Trial 01 summary
Hyperparameters:
DO: 0.4
rs: 0.004
optimizer: sgd
Score: 0.7414555549621582

Trial 02 sum

Guardardo de modelo

In [ ]:
fc_best_model.save('/content/salidas/fc_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Modelo con tunning 2

In [ ]:
#### función con definicion de hiperparámetros a afinar
hp = kt.HyperParameters()
def build_model(hp):

    dropout_rate=hp.Float('DO', min_value=0.1, max_value= 0.5, step=0.05)
    reg_strength = hp.Float("rs", min_value=0.001, max_value=0.005, step=0.001)
    optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop']) ### en el contexto no se debería afinar
    activation_opt = hp.Choice('activation', ['tanh', 'leaky_relu', 'relu'])  # la función de activación
    hp_learning_rate = hp.Choice('learning_rate', values=[0.1, 0.01, 0.001, 0.0001])

    #filtros1 = hp.Int("filtros1", min_value=16, max_value=128, step=32)
    #filtros2 = hp.Int("filtros2", min_value=32, max_value=128, step=32)
    #filtros3 = hp.Int("filtros3", min_value=64, max_value=128, step=32)

    ####hp.Int
    ####hp.Choice


    model= tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation=activation_opt, input_shape=x_train.shape[1:], kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation=activation_opt, kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation=activation_opt, kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])



    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate)
    else:
        opt = tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate)

    model.compile(
        optimizer=opt, loss="binary_crossentropy",  metrics=["Precision"],
    )


    return model



###########

tuner = kt.RandomSearch(
    hypermodel=build_model,
    hyperparameters=hp,
    tune_new_entries=True,
    objective=kt.Objective("precision", direction="max"),
    max_trials=10,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)


tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test), batch_size=100)

fc_best_model = tuner.get_best_models(num_models=1)[0]
tuner.results_summary()
fc_best_model.summary()



Trial 10 Complete [00h 06m 26s]
precision: 0.6182500720024109

Best precision So Far: 0.9268389940261841
Total elapsed time: 01h 05m 23s
Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="precision", direction="max")

Trial 02 summary
Hyperparameters:
DO: 0.4
rs: 0.005
optimizer: adam
activation: relu
learning_rate: 0.0001
Score: 0.9268389940261841

Trial 00 summary
Hyperparameters:
DO: 0.30000000000000004
rs: 0.004
optimizer: adam
activation: relu
learning_rate: 0.0001
Score: 0.9267516136169434

Trial 06 summary
Hyperparameters:
DO: 0.45000000000000007
rs: 0.003
optimizer: rmsprop
activation: tanh
learning_rate: 0.0001
Score: 0.890782356262207

Trial 07 summary
Hyperparameters:
DO: 0.5
rs: 0.001
optimizer: sgd
activation: tanh
learning_rate: 0.01
Score: 0.8858984112739563

Trial 04 summary
Hyperparameters:
DO: 0.35
rs: 0.001
optimizer: sgd
activation: leaky_relu
learning_rate: 0.01
Score: 0.8682724833488464

Trial 01 summary
Hyperparameters:
DO: 0.2
rs

 dropout (Dropout)           (None, 74, 74, 16)        0         


 conv2d_1 (Conv2D)           (None, 72, 72, 32)        4640      


 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 32)        0         


 g2D)                                                            


 dropout_1 (Dropout)         (None, 36, 36, 32)        0         


 flatten (Flatten)           (None, 41472)             0         


 dense (Dense)               (None, 64)                2654272   


 dropout_2 (Dropout)         (None, 64)                0         


 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2659425 (10.14 MB)
Trainable params: 2659425 (10.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Guardado de modelo


In [ ]:

#################### Mejor redes ##############
fc_best_model.save('/content/salidas/fc_model+.h5')
#cnn_model.save('salidas\\cnn_model.h5')


Para este modelo se balancearon las clases con la funcion SMOTEENN

In [ ]:
x_train.shape

(2870, 150, 150, 3)

In [ ]:
np.unique(y_train,return_counts=True)

(array([0, 1], dtype=int32), array([ 395, 2475]))

In [ ]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=42)
x_train_res=x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2]*x_train.shape[3])
X_train_resampled, y_train_resampled = smote_enn.fit_resample(x_train_res, y_train)

In [ ]:
X_train_resampled.shape

(4362, 67500)

In [ ]:
np.unique(y_train_resampled,return_counts=True)

(array([0, 1], dtype=int32), array([2384, 2097]))

In [ ]:
x_train_r = X_train_resampled.reshape((X_train_resampled.shape[0], x_train.shape[1], x_train.shape[2], x_train.shape[3]))



Modelo con tunning 3

In [ ]:
#### función con definicion de hiperparámetros a afinar
hp = kt.HyperParameters()
def build_model(hp):

    dropout_rate=hp.Float('DO', min_value=0.1, max_value= 0.5, step=0.05)
    reg_strength = hp.Float("rs", min_value=0.001, max_value=0.005, step=0.001)
    optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop']) ### en el contexto no se debería afinar
    activation_opt = hp.Choice('activation', ['tanh', 'leaky_relu', 'relu'])  # la función de activación
    hp_learning_rate = hp.Choice('learning_rate', values=[0.1, 0.01, 0.001, 0.0001])

    #filtros1 = hp.Int("filtros1", min_value=16, max_value=128, step=32)
    #filtros2 = hp.Int("filtros2", min_value=32, max_value=128, step=32)
    #filtros3 = hp.Int("filtros3", min_value=64, max_value=128, step=32)

    ####hp.Int
    ####hp.Choice


    model= tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation=activation_opt, input_shape=x_train.shape[1:], kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation=activation_opt, kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation=activation_opt, kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])



    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate)
    else:
        opt = tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate)

    model.compile(
        optimizer=opt, loss="binary_crossentropy",  metrics=["Precision"],
    )


    return model



###########

tuner = kt.RandomSearch(
    hypermodel=build_model,
    hyperparameters=hp,
    tune_new_entries=True,
    objective=kt.Objective("precision", direction="max"),
    max_trials=10,
    overwrite=True,
    directory="my_dir",
    project_name="tunnig2",
)


tuner.search(x_train_r, y_train_resampled, epochs=5, validation_data=(x_test, y_test), batch_size=100)

fc_best_model = tuner.get_best_models(num_models=1)[0]
tuner.results_summary()
fc_best_model.summary()


Trial 10 Complete [00h 05m 25s]
precision: 0.9043062329292297

Best precision So Far: 0.9781990647315979
Total elapsed time: 00h 44m 15s
Results summary
Results in my_dir/tunnig2
Showing 10 best trials
Objective(name="precision", direction="max")

Trial 01 summary
Hyperparameters:
DO: 0.1
rs: 0.003
optimizer: adam
activation: leaky_relu
learning_rate: 0.001
Score: 0.9781990647315979

Trial 02 summary
Hyperparameters:
DO: 0.25
rs: 0.002
optimizer: rmsprop
activation: leaky_relu
learning_rate: 0.001
Score: 0.967048704624176

Trial 08 summary
Hyperparameters:
DO: 0.30000000000000004
rs: 0.003
optimizer: sgd
activation: leaky_relu
learning_rate: 0.1
Score: 0.9614652991294861

Trial 06 summary
Hyperparameters:
DO: 0.45000000000000007
rs: 0.002
optimizer: rmsprop
activation: relu
learning_rate: 0.001
Score: 0.9497367143630981

Trial 07 summary
Hyperparameters:
DO: 0.45000000000000007
rs: 0.003
optimizer: adam
activation: leaky_relu
learning_rate: 0.01
Score: 0.9305292963981628

Trial 00 summ

Guardado de modelo

In [ ]:
fc_best_model.save('/content/salidas/fc_modelres.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Modelo con tunning 4

In [ ]:
#### función con definicion de hiperparámetros a afinar
hp = kt.HyperParameters()
def build_model(hp):

    dropout_rate=hp.Float('DO', min_value=0.1, max_value= 0.5, step=0.05)
    reg_strength = hp.Float("rs", min_value=0.001, max_value=0.005, step=0.001)
    optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop']) ### en el contexto no se debería afinar
    activation_opt = hp.Choice('activation', ['tanh', 'leaky_relu', 'relu'])  # la función de activación
    hp_learning_rate = hp.Choice('learning_rate', values=[0.1, 0.01, 0.001, 0.0001])

    #filtros1 = hp.Int("filtros1", min_value=16, max_value=128, step=32)
    #filtros2 = hp.Int("filtros2", min_value=32, max_value=128, step=32)
    #filtros3 = hp.Int("filtros3", min_value=64, max_value=128, step=32)

    ####hp.Int
    ####hp.Choice


    model= tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation=activation_opt, input_shape=x_train.shape[1:], kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation=activation_opt, kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation=activation_opt, kernel_regularizer=tf.keras.regularizers.l2(reg_strength)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])



    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate)
    else:
        opt = tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate)

    model.compile(
        optimizer=opt, loss="binary_crossentropy",  metrics=["Recall", "Precision", "Accuracy"],
    )


    return model



###########

tuner = kt.RandomSearch(
    hypermodel=build_model,
    hyperparameters=hp,
    tune_new_entries=True,
    objective=kt.Objective("precision", direction="max"),
    max_trials=10,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)


tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test), batch_size=100)

fc_best_model = tuner.get_best_models(num_models=1)[0]
tuner.results_summary()
fc_best_model.summary()



Trial 10 Complete [00h 04m 25s]
precision: 0.924091100692749

Best precision So Far: 0.9565393924713135
Total elapsed time: 00h 45m 55s
Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="precision", direction="max")

Trial 04 summary
Hyperparameters:
DO: 0.1
rs: 0.001
optimizer: rmsprop
activation: tanh
learning_rate: 0.001
Score: 0.9565393924713135

Trial 05 summary
Hyperparameters:
DO: 0.35
rs: 0.002
optimizer: adam
activation: leaky_relu
learning_rate: 0.001
Score: 0.9494787454605103

Trial 08 summary
Hyperparameters:
DO: 0.2
rs: 0.004
optimizer: rmsprop
activation: tanh
learning_rate: 0.001
Score: 0.9388504028320312

Trial 09 summary
Hyperparameters:
DO: 0.35
rs: 0.002
optimizer: sgd
activation: relu
learning_rate: 0.1
Score: 0.924091100692749

Trial 01 summary
Hyperparameters:
DO: 0.35
rs: 0.003
optimizer: sgd
activation: relu
learning_rate: 0.1
Score: 0.8841033577919006

Trial 07 summary
Hyperparameters:
DO: 0.2
rs: 0.005
optimizer: sgd
activation

Guardado de modelo

In [ ]:
fc_best_model.save('/content/salidas/fc_model4.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
